In [1]:
reset()
%display latex
Partitions.options.latex="list"
load("setup.sage")

We try to get the Verlinde-Segre correspondence for $\mathbb{C}^2$.

Over $\mathbb{C}^2$, the equivariant cohomology ring is $\mathbb{Z}[t_1,t_2]/(t_1+t_2)=\mathbb{Z}[t]$. The equivariant K-theory ring is $\mathbb{Z}[x^{\pm1}]$. The tangent space at $Z_\pi$ is
$$Q+\bar Q-Q\bar Q(1-x)(1-1/x) $$
where $Q$ is the character of partition $\pi$.

The Segre number is 
$$\int_{[X^{[n]}]} c(\alpha^{[n]})=\sum_{|\pi|=n}e(-T|_Z)c(\alpha^{[n]}|_Z) $$

The Verlinde number is 
$$V_n(\alpha)= \chi(\det (\alpha^{[n]}))=\int_{[X^{[n]}]}\text{td}(T)\text{ch}\left( \det (\alpha^{[n]})\right)$$
$$=\sum_{|\pi|=n}e(-T|_Z)\text{td}(T|_Z)\text{ch}\left( \det (\alpha^{[n]})|_Z\right)$$
$$=\sum_{|\pi|=n}\frac{1}{\text{ch}(\wedge T^\vee|_Z)}\text{ch}\left( \det (\alpha^{[n]})|_Z\right) $$
where ${\text{ch}(\wedge x^\vee)}=1/(1-e^{-x})$

In [2]:
KT_vars = ['x','y']
KT_ring = LaurentPolynomialRing(QQ, KT_vars)

coho_vars = ['t','m']
coho_ring = LaurentPolynomialRing(QQ, coho_vars)

x,y=KT_ring.gens()
t,m = coho_ring.gens()
b=var('b')

def character(p,x=x,y=1/x):
    return sum(y**i * x**j for i, j in p.cells())

def character_vb(p, w):
    Q = character(p)
    return sum(Q * x^(w[i]) for i in range(len(w)))

def tangent(p):
    q=character(p,x,1/x)
    q_inv=character(p,1/x,x)
    return q+q_inv-q*q_inv*(1-x)*(1-1/x)



def from_monomial(m):
    return sum(m.degree(KT_ring.gen(i)) * coho_ring.gen(i) for i in range(len(coho_ring.gens())))

def from_character(f):
    numer, denom = [], []

    for coeff, mon in list(f):
        wt = from_monomial(mon)
        if coeff > 0:
            numer += (coeff) * [wt]
        elif coeff < 0:
            denom += (-coeff) * [wt]
    return numer, denom
    
def top_chern(f):
    numer, denom = from_character(f)
    return coho_ring.fraction_field()(prod(numer) / prod(denom))
    
def total_chern(f):
    numer, denom = from_character(f)
    new_numer=[i+1 for i in numer]
    new_denom=[i+1 for i in denom]
    return coho_ring.fraction_field()(prod(new_numer+[1]) / (prod(new_denom)))

def Nek(f, check=False):
    R = f.parent()
    numer, denom = R.one(), R.one()
    for exp, coeff in f.dict().items():
        if exp.is_constant():
            term = R.zero()
        else:
            term = R({exp: 1, exp.emul(0): -1})
        if coeff > 0:
            numer *= term ** (coeff)
        elif coeff < 0:
            denom *= term ** (-coeff)
    return R.fraction_field()(numer) / denom
    
def dual(f, check=False):
    R = f.parent()
    g=R.zero()
    for exp, coeff in f.dict().items():
        g+= R({exp.emul(-1): coeff})
    return g

def highest_degree_y(f):
    R = f.parent()
    max_deg=0
    for exp, coeff in f.dict().items():
        if exp[1]>max_deg:
            max_deg = exp[1]
    term=R.zero()
    for exp, coeff in f.dict().items():
        if exp[1]==max_deg:
            term+=R({exp:coeff})
    return term

def lowest_degree_y(f):
        R = f.parent()
        min_deg=0
        for exp, coeff in f.dict().items():
            if exp[4]<min_deg:
                min_deg = exp[4]
        term=R.zero()
        for exp, coeff in f.dict().items():
            if exp[4]==min_deg:
                term+=R({exp:coeff})
        return term
    

In [4]:
def Total_Chern(w,n):
    return sum(top_chern(-tangent(p))*total_chern(character_vb(p,w)) for p in Partitions(n))

In [ ]:
W=(1,2,-2,3)
N=4

tc= Total_Chern(W,N)(t=b*t)
taylc = tc.taylor(b,0,0)
taylc

In [53]:
def Verlinde(w,n):
    return sum(KTheory.measure_unsymmetrized(tangent(p))* 
               KTheory.determinant(character_vb(p, w))
               for p in Partitions(n))


In [67]:
verl = Verlinde(W,N)(exp(t*b),0)
tayl = verl.taylor(b,0,0)
tayl

84844393/1296/(b*t) + 1932564959/145152/(b^2*t^2) + 886511/360/(b^3*t^3) + 7068877/17280/(b^4*t^4) + 533/9/(b^5*t^5) + 1045/144/(b^6*t^6) + 2/3/(b^7*t^7) + 1/24/(b^8*t^8) + 25612326139717/87091200

In [59]:
pretty(tayl-tc)

1/1440*(551*b^2*t^2 + 1080*b*t + 780)/(b^2*t^2)

Nekrasov genus is

$$\sum_{|\pi|=n}\frac{1}{\text{ch}(\wedge T^\vee|_Z)}\text{ch}\left(\frac{\wedge}{\det} \alpha^{[n]}|_Z\otimes y^{-1}\right)$$

where 

$$\text{ch}\left(\frac{\wedge}{\det} L^\vee\right)=e^{c_1(L)}-1$$

so

$$\mathcal{N}_y(x)=y/x-1$$

In [319]:
def Nek_Genus_dual(w,n):
    return sum(KTheory.measure_unsymmetrized(tangent(p))* 
               Nek(dual(character_vb(p, w))*y)
               for p in Partitions(n))

In [344]:
W_dual= tuple(-w for w in W)
nek = (highest_degree_y(Nek_Genus(W_dual,N).numerator())(y=1)/Nek_Genus(W_dual,N).denominator())
nek

(x^24 + x^18 + 2*x^17 + 3*x^16 + 2*x^15 + 2*x^14 + 2*x^12 + 2*x^10 + 2*x^9 + 3*x^8 + 2*x^7 + x^6 + 1)/(x^22 - 2*x^21 - x^20 + 2*x^19 + x^18 + 4*x^17 - 4*x^16 - 4*x^15 - 2*x^14 + 10*x^12 - 2*x^10 - 4*x^9 - 4*x^8 + 4*x^7 + x^6 + 2*x^5 - x^4 - 2*x^3 + x^2)

In [342]:
Verlinde(W,N)

(x^24 + x^18 + 2*x^17 + 3*x^16 + 2*x^15 + 2*x^14 + 2*x^12 + 2*x^10 + 2*x^9 + 3*x^8 + 2*x^7 + x^6 + 1)/(x^22 - 2*x^21 - x^20 + 2*x^19 + x^18 + 4*x^17 - 4*x^16 - 4*x^15 - 2*x^14 + 10*x^12 - 2*x^10 - 4*x^9 - 4*x^8 + 4*x^7 + x^6 + 2*x^5 - x^4 - 2*x^3 + x^2)

We see Verlinde number of $\alpha$ is max degree in $y$ of Nekrasov genus of $\alpha^\vee$.